# Generate Static LocusZoom
- **Author** - Frank Grenn
- **Date Started** - April 2020
- **Quick Description:** code to generate locus zoom pngs
- **Data:** 
[Static Locus Zoom](http://locuszoom.org/)  



In [ ]:
import pandas as pd

In [ ]:
DATADIR = "$PATH/AppDataProcessing"
WRKDIR = f"{DATADIR}/locuszoom"

## Reformat the summary stats file as needed
need a snp identifier column and a p value column

In [ ]:
sum_stats = pd.read_csv(f"{DATADIR}/asiangwas_sumstats_harmonized.csv")
print(sum_stats.shape)
print(sum_stats.head())

## Generate the hitSpec file

In [ ]:
risk_vars = ['rs246814','rs9638616']
hitSpec_file_name = f"{WRKDIR}/hitSpec_2MB_flank_asian_gwas.txt"

In [ ]:
#cols: snp	chr	start	end	flank	run	m2zargs
hitSpec_df = pd.DataFrame()
hitSpec_df['snp']=risk_vars
hitSpec_df['chr']='NA'
hitSpec_df['start']='NA'
hitSpec_df['end']='NA'
hitSpec_df['flank']='2MB'
hitSpec_df['run']='yes'
hitSpec_df['m2zargs']='height=10 rfrows=20 showRecomb=T geneFontSize=.4'

hitSpec_df.to_csv(hitSpec_file_name,sep='\t',index=None)

## Generate batch file

In [ ]:
population = "ASN"#"EUR"
formatted_sum_stats=f"{DATADIR}/asiangwas_sumstats_harmonized.csv"
delim = 'comma'#'space'

In [ ]:
with open(f"{WRKDIR}/run_locus_zoom_batch.sh","w") as batch_file:
    batch_file.write(f"#!/bin/bash\n\
module load locuszoom\n\
locuszoom --metal {formatted_sum_stats} --pvalcol P --markercol RSID --hitspec {hitSpec_file_name} --delim {delim} --pop {population} --build hg19 --source 1000G_March2012 --plotonly\n\
")
batch_file.close()

In [ ]:
print(f"sbatch {WRKDIR}/run_locus_zoom_batch.sh")

### move the generated files somewhere else if needed

In [ ]:
!find . -name '*.pdf' -exec mv {} LocusZoomPdfs/ \;

## Convert the pdf to png

In [ ]:
%%bash
## GWAS Locus Browser Locus Zoom Scripts
#- **Author** - Frank Grenn
#- **Date Started** - June 2019
#- **Quick Description:** script to convert locus zoom pdfs to pngs

#!/bin/bash

module load ImageMagick

for fullfilename in "LocusZoomPdfs"/*; do
	echo "$fullfilename"
	filename=$(basename -- "$fullfilename")
	filenamenoext="${filename%.*}"
	
	echo "$filenamenoext"
	
	snp="${filenamenoext:7}"
	echo "$snp"
	
	magick convert -density 300 -depth 8 -quality 85 "${fullfilename}[0]" "LocusZoomPngs/${snp}.png"
done
